In [ ]:

# coding: utf-8

# In[2]:


# Make directed from mentionee --> mentioner

from pymongo import MongoClient
import tweepy
from tweepy import AppAuthHandler
import seaborn as sns
import json
import pandas as pd
from igraph import *
get_ipython().run_line_magic('matplotlib', 'inline')

client = MongoClient()
db = client.mumbai_tweets_noloc_edited
collection = db.twitter_collection


source = {}
target = {}
edges_list = []
names = []

tweets_iterator = collection.find()

for tweet in tweets_iterator[0:10]:
    
    try:
        source = {'screen_name':tweet['user']['screen_name'].lower(),'id':tweet['user']['id']}
        
        for i in range(0,len(tweet['entities']['user_mentions'])):
            target = {'screen_name': tweet['entities']['user_mentions'][i]['name'].lower(), 
                      'id':tweet['entities']['user_mentions'][0]['id']}
            edges_list.append({'source':source,'target':target})

        
            names.append(tweet['user']['screen_name'].lower())
            names.append(tweet['entities']['user_mentions'][i]['name'].lower())
        
    except:
        pass



# In[3]:


from sklearn import preprocessing


le = preprocessing.LabelEncoder()
le.fit(names)


# In[4]:


edges_id = []
for i in edges_list:
    i['source']['id'] = int(le.transform([i['source']['screen_name']]))
    i['target']['id'] = int(le.transform([i['target']['screen_name']]))
    
    edges_id.append((i['source']['id'],i['target']['id']))


# In[5]:


g = Graph(edges_id)

# to_delete_ids = [v.index for v in g.vs if v.degree() <5]
# g.delete_vertices(to_delete_ids)

# E=[e.tuple for e in g.es]# list of edges


# In[6]:


to_delete_ids = [v.index for v in g.vs if v.degree() == 1]
g.delete_vertices(to_delete_ids)


# In[7]:


g.degree()


# In[8]:


E=[e.tuple for e in g.es]# list of edges


# In[10]:


layt = g.layout('kk')


# In[11]:


N = len(layt)


# In[12]:


N


# In[13]:


# Adding names to nodes

names_sorted = []
for i in range(0,N):
    names_sorted.append(le.inverse_transform(i))   

le.transform(names_sorted)

g.vs["screen_name"] = names_sorted




# In[14]:


len(names_sorted)


# In[15]:





labels=list(g.vs['screen_name'])



# In[16]:


import plotly
from plotly.graph_objs import *

Xn=[layt[k][0] for k in range(N)]
Yn=[layt[k][1] for k in range(N)]
Xe=[]
Ye=[]
for e in E:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]
    Ye+=[layt[e[0]][1],layt[e[1]][1], None] 
    
trace1=Scatter(x=Xe,
               y=Ye,
               mode='lines',
               line=Line(color='rgb(210,210,210)', width=1),
               hoverinfo='none'
               )
trace2=Scatter(x=Xn,
               y=Yn,
               mode='markers',
               name='ntw',
               marker=Marker(symbol='dot',
                             size=5, 
                             color='#6959CD',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )

width=800
height=800
layout=Layout(title= "Coauthorship network of scientists working on network theory and experiment"+              "<br> Data source: <a href='https://networkdata.ics.uci.edu/data.php?id=11'> [1]</a>",  
    font= Font(size=12),
    showlegend=False,
    autosize=False,
    width=width,
    height=height,
    xaxis=XAxis(axis),
    yaxis=YAxis(axis),          
    margin=Margin(
        l=40,
        r=40,
        b=85,
        t=100,
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False, 
            text='This igraph.Graph has the Kamada-Kawai layout',  
            xref='paper',     
            yref='paper',     
            x=0,  
            y=-0.1,  
            xanchor='left',   
            yanchor='bottom',  
            font=Font(
            size=14 
            )     
            )
        ]),           
    )

data=Data([trace1, trace2])
fig=Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='Coautorship-network-igraph') 

